In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

dbutils.fs.unmount("/mnt/0a8597384a69-bucket")
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a8597384a69-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/0a8597384a69-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)


/mnt/0a8597384a69-bucket has been unmounted.
Out[114]: True

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/0a8597384a69-bucket/topics/0a8597384a69.{}/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("pin"))
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("geo"))
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("user"))
# Display Spark dataframe to check its content

display(df_pin)


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,1,500k,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,2418,image,HikenDip,Local save in /data/christmas,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,da8745a6-5160-46c4-877d-181d50a729fd
home-decor,"Holiday mantle decor, Christmas decor, metallic mercury glass style Christmas trees, eucalyptus vine, evergreen pine branches, white neutral holiday decor, cozy mantle for the h…",1,83k,https://i.pinimg.com/originals/9d/82/1a/9d821a80acd8f90c16454e978bd9b115.jpg,6145,image,Stylin by Aylin,Local save in /data/home-decor,"Winter Home Decor,Christmas Living Room Decor,Living Room Decor Cozy,Christmas Decor,Cozy Fireplace,Rustic Fireplace Decor,Fireplace Decorations,Rustic Room,House Decorations",HOLIDAY MANTLE DECOR - @AMAZON & @TARGET FINDS,82e13a07-db99-43a3-b1c0-89a4b75821da
education,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",1,25k,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,4315,image,Math Giraffe,Local save in /data/education,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",Podcasts for Teachers or Parents of Teenagers,21b59ba9-829d-4c33-8c27-4cd4c56d26b8
event-planning,Sacramento California Wedding 2 Chic Events & Design Jodi Yorston Photography Wilson Vineyards Barn Miosa Couture Yellow Barn Vineyard Outdoor Candles DIY,1,6M,https://i.pinimg.com/originals/7e/45/90/7e45905fefa36347e83333fd6d091140.jpg,5069,image,Style Me Pretty,Local save in /data/event-planning,"60th Anniversary Parties,Anniversary Decorations,Golden Anniversary,25th Wedding Anniversary,Anniversary Pictures,Anniversary Ideas,Birthday Decorations,Event Planning Design,Event Design",The Vault: Curated & Refined Wedding Inspiration,b75b6f87-deb3-444f-b29e-ce9161b2df49
mens-fashion,"Black And Brown Outfits You Need To Try This Fall Season. The trick with black and brown outfits is to know how to combine colors, fabrics, and shapes to get a cohesive look. S

In [ ]:
from pyspark.sql.functions import col,when
from pyspark.sql.functions import regexp_extract

df_pin = df_pin.withColumn("description", when(col("description").contains("No description available"), "None").otherwise(col("description")))
df_pin = df_pin.withColumn("image_src", when(col("image_src").contains("Image src error"), "None").otherwise(col("image_src")))
df_pin = df_pin.withColumn("follower_count", when(col("follower_count").contains("User Info Error"), "None").otherwise(col("follower_count")))
df_pin = df_pin.withColumn("follower_count", regexp_replace(df_pin["follower_count"], "M", "000000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace(df_pin["follower_count"], "k", "000"))
df_pin = df_pin.withColumn("downloaded",col("downloaded").cast("int"))
df_pin = df_pin.withColumn("follower_count",col("follower_count").cast("int"))
df_pin = df_pin.withColumn("index",col("index").cast("int"))
df_pin = df_pin.withColumnRenamed("index","ind")
df_pin = df_pin.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")
display(df_pin)

print(df_pin.select('is_image_or_video').distinct().collect())

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
3156,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4,Handprint Reindeer Ornaments - Crafty Morning,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",892000,Michelle {CraftyMorning.com},"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",image,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,Local save in /data/diy-and-crafts,diy-and-crafts
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",26000,"Living Low Key | Save Money, Make Money, & Frugal Living","Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",image,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,Local save in /data/finance,finance
2418,da8745a6-5160-46c4-877d-181d50a729fd,100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,500000,HikenDip,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",image,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,Local save in /data/christmas,christmas
6145,82e13a07-db99-43a3-b1c0-89a4b75821da,HOLIDAY MANTLE DECOR - @AMAZON & @TARGET FINDS,"Holiday mantle decor, Christmas decor, metallic mercury glass style Christmas trees, eucalyptus vine, evergreen pine branches, white neutral holiday decor, cozy mantle for the h…",83000,Stylin by Aylin,"Winter Home Decor,Christmas Living Room Decor,Living Room Decor Cozy,Christmas Decor,Cozy Fireplace,Rustic Fireplace Decor,Fireplace Decorations,Rustic Room,House Decorations",image,https://i.pinimg.com/originals/9d/82/1a/9d821a80acd8f90c16454e978bd9b115.jpg,Local save in /data/home-decor,home-decor
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",25000,Math Giraffe,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,Local save in /data/education,education
5069,b75b6f87-deb3-444f-b29e-ce9161b2df49,The Vault: Curated & Refined Wedding Inspiration,Sacramento California Wedding 2 Chic Events & Design Jodi Yorston Photography Wilson Vineyards Barn Miosa Couture Yellow Barn Vineyard Outdoor Candles DIY,6000000,Style Me Pretty,"60th Anniversary Parties,Anniversary Decorations,Golden Anniversary,25th Wedding Anniversary,Anniversary Pictures,Anniversary Ideas,Birthday Decorations,Event Planning Design,Event Design",image,https://i.pinimg.com/originals/7e/45/90/7e45905fefa36347e83333fd6d091140.jpg,Local save in /data/event-planning,event-planning
7343,b8c63a95-6fe8-4d7f-9455-1eef3462fee0,"This Fashion Rule Is Nonsense, And We Should All Forget About It","Black And Brown Outfits You Need To Try This Fall Season. The trick with black and brown o

[Row(is_image_or_video='image'), Row(is_image_or_video='multi-video(story page format)'), Row(is_image_or_video='video')]

In [ ]:
from pyspark.sql.functions import col,when
from pyspark.sql.functions import regexp_extract

# df_geo = df_geo.select("index", "country", array(col("longitude"), col("latitude")).alias("coordinates"), "timestamp")
df_geo = df_geo.withColumn("timestamp", col("timestamp").cast("timestamp"))
df_geo = df_geo.withColumnRenamed("index","ind")

display(df_geo)

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59.000+0000
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50.000+0000
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08T22:53:09.000+0000
French Guiana,8304,-28.8852,-164.87,2019-09-13T04:50:29.000+0000
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15T03:51:28.000+0000
Azerbaijan,5069,-63.0063,-157.474,2021-03-20T09:32:44.000+0000
Mozambique,6145,-65.9079,-143.845,2019-12-05T02:09:44.000+0000
Bangladesh,428,-83.4105,-150.788,2020-09-27T18:46:41.000+0000
Australia,7343,-65.4428,-161.684,2018-08-06T12:17:07.000+0000
Bulgaria,5494,-82.6768,-129.202,2021-07-21T02:02:35.000+0000


In [ ]:
df_user = df_user.withColumn("user_name", concat(col("first_name"), lit(" "), col("last_name")))
df_user = df_user.withColumn("date_joined", col("date_joined").cast("timestamp"))
df_user = df_user.withColumnRenamed("index","ind")
df_user = df_user.select("ind", "user_name", "age", "date_joined")
display(df_user)

ind,user_name,age,date_joined
8731,Andrea Alexander,21,2015-11-10T09:27:42.000+0000
4315,Michelle Prince,36,2015-12-20T16:38:13.000+0000
10625,Christian Lang,32,2017-10-10T20:09:33.000+0000
1313,Brittany Jones,32,2016-04-02T03:51:23.000+0000
10794,Thomas Turner,34,2016-12-22T00:02:02.000+0000
2959,David Griffith,20,2016-01-07T19:49:22.000+0000
9875,Brendan Joseph,26,2015-12-20T10:28:00.000+0000
8304,Charles Berry,25,2015-12-28T04:21:39.000+0000
6063,Corey Andrews,23,2015-11-25T13:36:22.000+0000
3454,Robert Murphy,48,2017-09-26T16:31:56.000+0000


In [ ]:
df_pin.createOrReplaceTempView("pin")
df_geo.createOrReplaceTempView("geo")
df_user.createOrReplaceTempView("user")

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
3156,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4,Handprint Reindeer Ornaments - Crafty Morning,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",892000,Michelle {CraftyMorning.com},"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",image,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,Local save in /data/diy-and-crafts,diy-and-crafts
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",26000,"Living Low Key | Save Money, Make Money, & Frugal Living","Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",image,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,Local save in /data/finance,finance
2418,da8745a6-5160-46c4-877d-181d50a729fd,100 DIY Christmas Centerpieces You'll Love To Decorate Your Home With For The Christmas Season - Hike n Dip,Here are the best DIY Christmas Centerpieces ideas perfect for your Christmas & holiday season home decor. From Christmas Vignettes to Table Centerpieces.,500000,HikenDip,"Farmhouse Christmas Decor,Rustic Christmas,Christmas Time,Vintage Christmas,Xmas,Primitive Christmas Crafts,Christmas Vignette,Indoor Christmas Decorations,Diy Christmas Ornaments",image,https://i.pinimg.com/originals/aa/6d/0f/aa6d0f44d7c1c96b998cb9aa6c4446b8.png,Local save in /data/christmas,christmas
6145,82e13a07-db99-43a3-b1c0-89a4b75821da,HOLIDAY MANTLE DECOR - @AMAZON & @TARGET FINDS,"Holiday mantle decor, Christmas decor, metallic mercury glass style Christmas trees, eucalyptus vine, evergreen pine branches, white neutral holiday decor, cozy mantle for the h…",83000,Stylin by Aylin,"Winter Home Decor,Christmas Living Room Decor,Living Room Decor Cozy,Christmas Decor,Cozy Fireplace,Rustic Fireplace Decor,Fireplace Decorations,Rustic Room,House Decorations",image,https://i.pinimg.com/originals/9d/82/1a/9d821a80acd8f90c16454e978bd9b115.jpg,Local save in /data/home-decor,home-decor
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",25000,Math Giraffe,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,Local save in /data/education,education
5069,b75b6f87-deb3-444f-b29e-ce9161b2df49,The Vault: Curated & Refined Wedding Inspiration,Sacramento California Wedding 2 Chic Events & Design Jodi Yorston Photography Wilson Vineyards Barn Miosa Couture Yellow Barn Vineyard Outdoor Candles DIY,6000000,Style Me Pretty,"60th Anniversary Parties,Anniversary Decorations,Golden Anniversary,25th Wedding Anniversary,Anniversary Pictures,Anniversary Ideas,Birthday Decorations,Event Planning Design,Event Design",image,https://i.pinimg.com/originals/7e/45/90/7e45905fefa36347e83333fd6d091140.jpg,Local save in /data/event-planning,event-planning
7343,b8c63a95-6fe8-4d7f-9455-1eef3462fee0,"This Fashion Rule Is Nonsense, And We Should All Forget About It","Black And Brown Outfits You Need To Try This Fall Season. The trick with black and brown o

In [ ]:
result = spark.sql("""SELECT country, category, category_count
                    FROM (
                        SELECT country, category, COUNT(*) AS category_count,
                            ROW_NUMBER() OVER (PARTITION BY country ORDER BY COUNT(*) DESC) AS rank
                        FROM geo 
                        JOIN pin ON pin.ind = geo.ind
                        GROUP BY country, category
                    ) 
                    WHERE rank = 1""")
display(result)

country,category,category_count
Albania,diy-and-crafts,1
Angola,diy-and-crafts,1
Anguilla,home-decor,1
Antarctica (the territory South of 60 deg S),christmas,1
Armenia,diy-and-crafts,2
Aruba,tattoos,1
Australia,mens-fashion,1
Azerbaijan,event-planning,1
Bangladesh,art,1
Barbados,travel,1


In [ ]:
result = spark.sql("""SELECT post_year, category, category_count
                    FROM (
                        SELECT year(timestamp) as post_year, category, COUNT(*) AS category_count,
                            ROW_NUMBER() OVER (PARTITION BY year(timestamp) ORDER BY COUNT(*) DESC) AS rank
                        FROM geo 
                        JOIN pin ON pin.ind = geo.ind
                        GROUP BY year(timestamp), category
                    ) 
                    WHERE rank = 1""")
display(result)


post_year,category,category_count
2018,diy-and-crafts,2
2019,diy-and-crafts,2
2020,art,1
2021,finance,2
2022,christmas,1


In [ ]:
result = spark.sql("""SELECT country, MAX(follower_count) as max_follower_count
                   FROM pin 
                   JOIN geo ON geo.ind = pin.ind
                   GROUP BY country""")

display(result)

country,max_follower_count
Anguilla,92000
Cambodia,6000
Bangladesh,3000000
Cocos (Keeling) Islands,437
Cote d'Ivoire,192000
Azerbaijan,6000000
Antarctica (the territory South of 60 deg S),500000
French Guiana,51000
Mozambique,83000
Australia,1000000


In [ ]:
result = spark.sql("""SELECT country,  MAX(follower_count) as max_follower_count
                   FROM pin
                   JOIN geo ON geo.ind = pin.ind
                   GROUP BY country
                   ORDER BY max_follower_count DESC
                   LIMIT 1""")

display(result)

country,max_follower_count
Azerbaijan,6000000


In [ ]:
result = spark.sql("""
    SELECT age_group, category, category_count
    FROM (
        SELECT age_group, category, category_count,
            ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY category_count DESC) AS rank
        FROM (
            SELECT
                CASE 
                    WHEN age BETWEEN 18 AND 24 THEN "18-24"
                    WHEN age BETWEEN 25 AND 35 THEN "25-35"
                    WHEN age BETWEEN 36 AND 50 THEN "36-50"
                    WHEN age > 50 THEN "+50"
                    ELSE "NONE" 
                END AS age_group,
                category,
                COUNT(*) AS category_count
            FROM user
            JOIN pin ON pin.ind = user.ind
            GROUP BY age_group, category
        )
    )
    WHERE rank = 1
""")

display(result)

age_group,category,category_count
18-24,diy-and-crafts,3
25-35,vehicles,2
36-50,education,1


In [ ]:
results = spark.sql("""
                    SELECT CASE 
                    WHEN age BETWEEN 18 AND 24 THEN "18-24"
                    WHEN age BETWEEN 25 AND 35 THEN "25-35"
                    WHEN age BETWEEN 36 AND 50 THEN "36-50"
                    WHEN age > 50 THEN "+50"
                    ELSE "NONE" 
                END AS age_group, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
                FROM user 
                JOIN pin on pin.ind = user.ind
                GROUP BY age_group
                    """)

display(results)

age_group,median_follower_count
36-50,6000.0
18-24,501000.0
25-35,47000.0


In [ ]:
results = spark.sql("""
                    SELECT year(date_joined) as join_year, COUNT(date_joined) as numbers_users_joined
                    FROM user 
                    GROUP by join_year
                    HAVING join_year BETWEEN 2015 and 2020
                    """)

display(results)

post_year,numbers_users_joined
2015,14
2016,6
2017,3


In [ ]:
results = spark.sql("""
                    SELECT year(date_joined) as join_year, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
                    FROM user 
                    JOIN pin on pin.ind = user.ind
                    GROUP by join_year
                    HAVING join_year BETWEEN 2015 and 2020
                    """)
display(results)

join_year,median_follower_count
2015,87500.0
2016,313000.0
2017,6000.0


In [ ]:
results = spark.sql("""
                    SELECT CASE 
                    WHEN age BETWEEN 18 AND 24 THEN "18-24"
                    WHEN age BETWEEN 25 AND 35 THEN "25-35"
                    WHEN age BETWEEN 36 AND 50 THEN "36-50"
                    WHEN age > 50 THEN "+50"
                    ELSE "NONE" 
                END AS age_group, year(date_joined) as join_year, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
                    FROM user 
                    JOIN pin on pin.ind = user.ind
                    GROUP by join_year, age_group
                    HAVING join_year BETWEEN 2015 and 2020
                    """)
                    
display(results)

age_group,join_year,median_follower_count
25-35,2015,51000.0
18-24,2016,751000.0
18-24,2015,500000.0
25-35,2016,83500.0
36-50,2017,6000.0
36-50,2015,12500.0
25-35,2017,8000.0
18-24,2017,940.0
